In [96]:
import cv2
import numpy as np
import time

def filtrar_color_amarillo(imagen):
    # Convertir la imagen a formato HSV (Hue, Saturation, Value)
    hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    
    # Definir el rango de colores amarillos en HSV
    lower_yellow = np.array([18, 100, 100])
    upper_yellow = np.array([32, 255, 255])
    
    # Aplicar el filtro para detectar solo el color amarillo
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # Crear una imagen binaria utilizando la máscara
    resultado = cv2.bitwise_and(imagen, imagen, mask=mask)
    
    return mask, resultado

def remove_small_particles(binary_image, min_area):
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a mask to store the filtered particles
    filtered_mask = np.zeros_like(binary_image)
    
    # Iterate through contours
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)
        
        # If the area is larger than the minimum area threshold, keep the contour
        if area >= min_area:
            cv2.drawContours(filtered_mask, [contour], -1, 255, cv2.FILLED)
    
    return filtered_mask





def remove_small_particles2(binary_image, min_area):
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create lists to store positions and areas of particles
    particle_positions = []
    particle_areas = []
    
    # Create a mask to store the filtered particles
    filtered_mask = np.zeros_like(binary_image)
    
    # Iterate through contours
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)
        
        # If the area is larger than the minimum area threshold, keep the contour
        if area >= min_area:
            # Get the centroid of the contour
            M = cv2.moments(contour)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            
            # Append position and area to the lists
            particle_positions.append([cx, cy])
            particle_areas.append(area)
            
            # Draw the contour on the filtered mask
            cv2.drawContours(filtered_mask, [contour], -1, 255, cv2.FILLED)
    
    # Convert lists to NumPy arrays
    particle_positions = np.array(particle_positions)
    particle_areas = np.array(particle_areas)
    
    return filtered_mask, particle_positions, particle_areas

def expand_binary_zone(binary_image, iterations=1):
    # Definir el kernel para la operación de dilatación
    kernel = np.ones((3, 3), np.uint8)
    
    # Aplicar la operación de dilatación
    expanded_image = cv2.dilate(binary_image, kernel, iterations=iterations)
    
    return expanded_image



def shrink_binary_zone(binary_image, iterations=1):
    # Define the kernel for the erosion operation
    kernel = np.ones((3, 3), np.uint8)
    
    # Apply the erosion operation
    shrunk_image = cv2.erode(binary_image, kernel, iterations=iterations)
    
    return shrunk_image

def filtering_1(imagen, mode=0,dif=3):

    mask_amarillo, resultado_amarillo = filtrar_color_amarillo(imagen)
    # Mostrar las imágenes

    # Set the minimum area threshold
    height, width = mask_amarillo.shape
    min_area_threshold = int(height*width*0.00005)

    # Remove small particles from the binary image
    filtered_image,areas,positions = remove_small_particles2(mask_amarillo, min_area_threshold)

    if mode!=0:
        cv2.imshow('Original', imagen)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imshow('Máscara Amarilla', mask_amarillo)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imshow('Filtered Binary Image', filtered_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    filtered_image=expand_binary_zone(filtered_image,dif)

    if mode!=0:
        cv2.imshow('Filtered Binary Image', filtered_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    filtered_image=shrink_binary_zone(filtered_image,dif)

    if mode!=0:
        cv2.imshow('Filtered Binary Image', filtered_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


    bordes = cv2.Canny(filtered_image, 100, 200)
    if mode!=0:
        cv2.imshow('Bordes', bordes)
        cv2.waitKey(0)
        #cv2.waitKey(0)
        cv2.destroyAllWindows()
    return bordes , filtered_image

def centroid_and_area(binary_image):
    # Encuentra los contornos en la imagen binaria
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    centroids = []
    areas = []
    
    # Itera sobre los contornos encontrados
    for contour in contours:
        # Calcula el centroide del contorno
        M = cv2.moments(contour)
        if M['m00'] != 0:  # Asegura que no hay división por cero
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            centroids.append((cx, cy))
        
        # Calcula el área del contorno
        area = cv2.contourArea(contour)
        areas.append(area)
    
    return centroids, areas




In [97]:
import cv2
import numpy as np
import math
def add_border(image, top, bottom, left, right):
    """
    Add borders of specified size to a binary image.

    Parameters:
        image: Input binary image.
        top: Size of the top border.
        bottom: Size of the bottom border.
        left: Size of the left border.
        right: Size of the right border.

    Returns:
        Image with borders added.
    """
    # Define border size for each side
    border_size = (top, bottom, left, right)

    # Create a border filled with zeros
    bordered_image = np.zeros((image.shape[0] + top + bottom, image.shape[1] + left + right), dtype=np.uint8)

    # Copy the original image to the center of the bordered image
    bordered_image[top:top + image.shape[0], left:left + image.shape[1]] = image

    return bordered_image



def linear_transform(image, transformation_matrix, output_size):
    transformed_image = cv2.warpAffine(image, transformation_matrix, output_size, flags=cv2.INTER_NEAREST)
    return transformed_image

def ScalRot_transformation(image,scale_x,scale_y,angle_a_deg):
    angle_a_rad = math.radians(angle_a_deg)

    # Calculate the transformation matrix
    transformation_matrix = np.float32([
        [scale_x * math.cos(angle_a_rad), -scale_y * math.sin(angle_a_rad), 30],
        [scale_x * math.sin(angle_a_rad), scale_y * math.cos(angle_a_rad), 0]
    ])
    # Get the shape of the input image
    height, width = image.shape[:2]
    # Calculate the size of the output image to accommodate all pixels
    corners = np.array([[0, 0], [width, 0], [0, height], [width, height]])
    transformed_corners = cv2.transform(np.array([corners]), transformation_matrix)[0]
    min_x, min_y = np.min(transformed_corners, axis=0)
    max_x, max_y = np.max(transformed_corners, axis=0)
    output_size = (int(max_x - min_x), int(max_y - min_y))
    # Update the transformation matrix to translate the image to keep it within bounds
    transformation_matrix[0, 2] -= min_x
    transformation_matrix[1, 2] -= min_y
    # Apply the affine transformation
    transformed_image = linear_transform(image, transformation_matrix, output_size)

    return transformed_image

In [98]:
import random
def Obj_fun(atributes,BinImage=0,BinKernel=0,mode=0):
    if (type(BinImage)==int or type(BinKernel)==int):
        print('there are no minaes')
        return 0
    [a,px,py,Ax,Ay]=atributes
    px=int(px)
    py=int(py)
    if mode!=0:
        cv2.imshow('ker Binary Image', BinKernel)
        cv2.imshow('img Binary Image', BinImage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


    NewKer=ScalRot_transformation(BinKernel,Ax,Ay,a)
    height, width = NewKer.shape

    if mode!=0:
        cv2.imshow('ker Binary Image', NewKer)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


    NewImg=add_border(BinImage,height+3,height+3,width+3,width+3)

    if mode!=0:
        cv2.imshow('img Binary Image', NewImg)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    NewImg = NewImg[py:py+height, px:px+width]
    #NewImg = cv2.resize(NewImg, (width, height), interpolation=cv2.INTER_NEAREST)

    if mode!=0:
        cv2.imshow('img Binary Image', NewImg)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    diff = cv2.bitwise_and(NewImg, NewKer)
    if mode!=0:
        cv2.imshow('diff Binary Image', diff)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return (2*np.sum(diff) - np.sum(NewImg))/(255*(Ax+Ay))
    
def rep_fun(atributes1,atributes2):
        return (atributes1 + atributes2)/2

def mut_fun(atributes,ImgH=0,ImgW=0,KerH=0,KerW=0):
        [a,px,py,Ax,Ay]=atributes
        
        amp = random.random()*0.5 + 0.75
        Ax=amp*Ax

        amp = random.random()*0.5 + 0.75
        Ay=amp*Ay

        step=random.randint(0, int(ImgW/5))-int(ImgW/10)
        px+=step

        step=random.randint(0, int(ImgH/5))-int(ImgH/10)
        py+=step

        step=random.random()*5 + -2.5
        a+=step


        if Ax<0.5:
            Ax=0.55
        elif Ax>ImgW/KerW*(0.8):
             Ax=(ImgW/KerW)*(0.8)

        if Ay<0.5:
            Ay=0.55

        elif Ay>ImgH/KerH*(0.8):
             Ay=(ImgH/KerH)*(0.8)

        if px > ImgW:
             px=ImgW
        elif px < 0:
             px=0

        if px > ImgH:
             px=ImgH
        elif py < 0:
             py=0

        if a >30:
             a=30
        elif a <-30:
             a=-30
        
        return [a,px,py,Ax,Ay]

def MakeaSeed(a,pos,n):
    dots=len(a)-1
    seed=np.zeros((n, 5))
    for i in range(0,n):
        ii=random.randint(0, dots)
        seed[i][:]=np.array([random.random()*5 + -2.5,pos[ii][0],pos[ii][1],np.sqrt(a[ii]),np.sqrt(a[ii])])
    return seed
    

In [99]:
import random
class GenAlg:
    def __init__(GA, Npop, Nvar,Ofun,Rfun,Mfun):
        GA.Npop=Npop
        GA.Nvar=Nvar

        GA.population = np.zeros((Npop, Nvar))
        GA.fitness=np.zeros(Npop)


        GA.ObjFun=Ofun
        GA.RepFun=Rfun
        GA.MutpFun=Mfun

        GA.temp=1
        GA.best=np.zeros(Nvar)
        GA.bestVal=-1000000000


    def FitEval(GA):
        for i in range(0,GA.Npop):
            GA.fitness[i]=GA.ObjFun(GA.population[i])
            if GA.fitness[i]>GA.bestVal:
                GA.bestVal=GA.fitness[i]
                GA.best=GA.population[i]


    def EstReprod(GA):
        GA.FitEval()
        # Apply temperature scaling to the logits
        GA.fitness = GA.fitness / GA.temp

        # Compute softmax probabilities
        exp_logits = np.exp(GA.fitness)
        GA.fitness = (exp_logits / np.sum(exp_logits))*GA.Npop

    def New_pop(GA):
        GA.EstReprod()
        old_Pop = GA.population
        i=0
        n=0
        for i in range(0,GA.Npop):
            while(GA.fitness[i]>=1):
                GA.fitness[i]=GA.fitness[i]-1
                GA.population[n]=old_Pop[i]
                n=n+1

        
        while(n<GA.Npop):
            new=random.randint(0, GA.Npop)-1
            GA.population[n]=old_Pop[new]
            n=n+1
        
        old_Pop = GA.population
        for i in range(0,GA.Npop):
            mate=random.randint(0, GA.Npop)-1
            GA.population[i]=GA.RepFun(GA.population[i],old_Pop[mate])
            GA.population[i]=GA.MutpFun(GA.population[i])


    def emulate_Ngen(GA,n):
        for i in range(0,n):
            print(i/n*100,'%')
            GA.New_pop()





    def setSeed(GA,seed):
        GA.population=seed

    def setRepFun(GA,rep):
        GA.RepFun=rep

    def setObjFun(GA,obj):
        GA.ObjFun=obj

    def setMutFun(GA,mut):
        GA.MutpFun=mut

    def setTemp(GA,t):
        GA.temp=t

        






            
    

In [100]:
for i in range(7, 7):  # Suponiendo que tienes imágenes de test1.jpg a test10.jpg
    # Formar el nombre de archivo
    filename = f'test{i}.jpg'

    # Cargar la imagen
    image = cv2.imread(filename)

    # Comprobar si la imagen se cargó correctamente
    if image is not None:
        filtering_1(image, mode=1)
    else:
        print(f'No se pudo cargar la imagen {filename}')

In [101]:
kernel_name = 'Rombo.jpg'
kerne = cv2.imread(kernel_name)
kernel,filt=filtering_1(kerne, dif=1)

img_name = 'test1.jpg'
img = cv2.imread(img_name)
img,filt=filtering_1(img)
p,a=centroid_and_area(filt)
population_size=len(a)*5

seed=MakeaSeed(a,p,population_size)
kh, kw = kernel.shape
ih, iw = img.shape

population=GenAlg(population_size,5, lambda x:  Obj_fun(x,BinImage=img,BinKernel=kernel),rep_fun, lambda x: mut_fun(x,ImgH=ih,ImgW=iw,KerH=kh,KerW=kw))
population.setTemp(0.5)
population.setSeed(seed)
population.emulate_Ngen(10)


0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %


In [107]:
population.best

array([1.52826117e-01, 3.21523202e+02, 2.60834068e+02, 6.47398590e+00,
       5.48532669e+00])

In [108]:
population.bestVal

0.0

In [109]:
Obj_fun(population.best,BinImage=img,BinKernel=kernel,mode=1)

0.0

In [105]:
len(a)

36

In [106]:
population.population


array([[-4.77619168e+00,  3.27822902e+02,  3.11233226e+02,
         6.09515155e+00,  4.43576661e+00],
       [-3.95588618e+00,  2.14162567e+02,  2.97377647e+02,
         8.92035398e+00,  5.98077694e+00],
       [-2.30284722e+00,  3.36439668e+02,  2.11551521e+02,
         5.67143173e+00,  4.63662996e+00],
       [ 1.71895037e+00,  2.93453924e+02,  3.69973388e+02,
         6.18591733e+00,  3.86776194e+00],
       [ 2.67940386e-01,  4.01654241e+02,  3.46173079e+02,
         7.45844190e+00,  5.51901238e+00],
       [-9.84906913e-01,  1.59929438e+02,  2.45359760e+02,
         7.99432403e+00,  3.26548640e+00],
       [ 1.31808821e+00,  1.48262751e+02,  3.30113542e+02,
         7.40628242e+00,  4.25479051e+00],
       [-2.48078300e+00,  1.70627515e+02,  3.11169936e+02,
         4.88130870e+00,  3.50874793e+00],
       [-2.48610559e+00,  2.40646342e+02,  1.45623475e+02,
         8.44544567e+00,  4.89260080e+00],
       [-2.39466969e+00,  2.92804292e+02,  2.93979580e+02,
         5.59864184e+00